# TP4 Hopfield

In [ ]:
from asciiHandler import asciiStringToLettersMatrix
from hopfield import printLetter,printLetterBipolar,printLetterBipolarMatrix,plotLetterMap
import numpy as np
import seaborn as sns
from string import ascii_lowercase
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
alphabet="""
 ###  ####   ###  ####  ##### #####  ###  #   # ##### ##### #   # #     #   #
#   # #   # #   # #   # #     #     #     #   #   #     #   #  #  #     ## ##
##### ####  #     #   # ####  ####  #  ## #####   #     #   ###   #     # # #
#   # #   # #   # #   # #     #     #   # #   #   #   # #   #  #  #     #   #
#   # ####   ###  ####  ##### #      ###  #   # ##### ###   #   # ##### #   #

#   #  ###  ####   ###  ####   ###  ##### #   # #   # #   # #   # #   # #####
##  # #   # #   # #   # #   # #       #   #   # #   # #   #  # #   # #     # 
# # # #   # ####  #   # ####   ###    #   #   #  # #  # # #   #     #     #  
#  ## #   # #     #  #  #   #     #   #   #   #  # #  ## ##  # #    #    #   
#   #  ###  #      ## # #   #  ###    #    ###    #   #   # #   #   #   #####"""

In [ ]:
#MOVE EACH ASCII TO A LETTER MATRIX
(alphabetASCII,alphabetASCIIBipolar)=asciiStringToLettersMatrix()

In [ ]:
for letterIndex in range(0,len(alphabetASCII)):
    print(printLetter(alphabetASCII,letterIndex))
    print(printLetterBipolar(alphabetASCIIBipolar,letterIndex))

In [ ]:
#FUNCION SIGNO
sgn = lambda x: -1 if x<=0 else 1
sgn_v= np.vectorize(sgn)

In [ ]:
#BUSCAR LETRAS MAS ORTOGONALES
ortogonality=np.zeros((len(alphabetASCIIBipolar),len(alphabetASCIIBipolar)))
alphabet = [c for c in ascii_lowercase]
print(np.dot(alphabetASCIIBipolar[1],alphabetASCIIBipolar[0]))
for i in range(0,len(alphabetASCIIBipolar)):
    for j in range(0,len(alphabetASCIIBipolar)):
        ortogonality[i][j]=np.dot(alphabetASCIIBipolar[i],alphabetASCIIBipolar[j])
df_cm = pd.DataFrame(ortogonality, index = [i for i in ascii_lowercase],
                  columns = [i for i in ascii_lowercase])
print(df_cm.head())
plt.figure(figsize = (15,9))
sns.heatmap(data=df_cm,cmap=sns.color_palette("vlag", as_cmap=True),annot=True)

#sns.heatmap(data=df_cm,cmap=sns.diverging_palette(250, 30, l=65, center="dark", as_cmap=True),annot=True)
plt.title("Producto interno entre")

In [ ]:
# CALCULAR 4 LETRAS MAS ORTOGONALES yMENOS ORTOGONALES
min_ortogonality_letters=[]
min_ortogonality=[25**6]
for i in range(0,len(alphabet)):
    for j in range(0,len(alphabet)):
        for k in range(0,len(alphabet)):
            for r in range(0,len(alphabet)):
                tot_ortogonality=ortogonality[i][j]*ortogonality[i][k]*ortogonality[i][r]*ortogonality[j][k]*ortogonality[j][r]*ortogonality[r][k]
                if(np.abs(tot_ortogonality)<=min_ortogonality[0] and i!=j and i!=k and i!=r and j!=k and j!=r and k!=r):
                    min_ortogonality_letters.insert(0,f"{ascii_lowercase[i]}{ascii_lowercase[j]}{ascii_lowercase[k]}{ascii_lowercase[r]}")
                    min_ortogonality.insert(0,np.abs(tot_ortogonality))
print(min_ortogonality_letters)
print(min_ortogonality)

In [ ]:
# AGARRAR 4 LETRAS
patterns_letters=['a','b','c','d']
patterns=[]
for i in range(0,len(patterns_letters)):
    patterns.append(alphabetASCIIBipolar[ascii_lowercase.index(patterns_letters[i])])
# CALCULAR LOS PESOS DE LA RED
W=np.zeros((25,25))
for pattern in patterns:
    w_aux=np.dot(np.transpose(np.matrix(pattern)),np.matrix(pattern))
    W=W + w_aux
division = np.vectorize(lambda m:m/25)
W= division(W)


In [ ]:
#VERIFICAR PATRONES
results=[]
iterations=[]
for p in range(0,len(patterns_letters)):
    results.append([])
    prevResult = sgn_v(np.matmul(W,patterns[p]))
    currentResult=None
    i=0
    results[p].append(prevResult)
    while currentResult is None or  not (prevResult==currentResult).all() :
        if currentResult is not  None:
            prevResult=currentResult
        currentResult=sgn_v(np.matmul(prevResult,W))
        results[p].append(currentResult)
        i+=1
    
    iterations.append(i)

#df = pd.DataFrame(data=[[np.reshape(result,(5,5)) for result in results]],index=patterns_letters)
##print(results[0][0])
print(results)
print(iterations)
print("MAX ITERATIONS===",max(iterations))
printLetterBipolarMatrix(results[0][0])
figure, axes = plt.subplots(max(iterations)+1, 4, sharex=True, figsize=(19,7))
for i in range(0,len(patterns)):
    plotLetterMap(patterns[i],axes=axes[0, i])
    for j in range(0,len(results[i])-1):
        plotLetterMap(results[i][j],axes=axes[j+1, i])